In [ ]:
# %% [markdown]
# # Предварительная обработка данных для машинного обучения на примере автомобильного датасета
# 
# ## Введение
# Машинное обучение — это процесс, в котором компьютер учится интерпретировать данные, принимать решения и делать прогнозы.
# Однако, если данные «грязные» (содержат пропуски, дубликаты, ошибки или аномальные значения), то результаты моделирования могут быть искажены.
# 
# В этом notebook рассмотрим **пример полной предобработки данных** на примере автомобильного набора данных `cars.csv`.
# 
# Этапы предобработки:
# - Проверка данных
# - Очистка данных
# - Обработка пропусков
# - Работа с аномалиями (outliers)
# - Кодирование категориальных переменных
# - Оптимизация и подготовка финального датафрейма

# %% 
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid", palette="Set2")

# %% [markdown]
# ## 1. Загрузка данных и первичный анализ

# %%
df = pd.read_csv("cars.csv")
print("Размер датафрейма:", df.shape)
df.head()

# %%
df.info()

# %%
df.describe(include="all")

# %% [markdown]
# **Вывод:**  
# В датасете 9 признаков и 301 наблюдение.  
# Есть числовые и категориальные переменные. Пропусков нет, но возможны выбросы или несоответствия.  

# %% [markdown]
# ## 2. Проверка на дубликаты и уникальные значения

# %%
print("Количество дубликатов:", df.duplicated().sum())
df.nunique()

# %%
# Удалим возможные дубликаты
df = df.drop_duplicates()
print("Размер после удаления дубликатов:", df.shape)

# %% [markdown]
# ## 3. Проверка категориальных переменных

# %%
cat_cols = ['Car_Name', 'Fuel_Type', 'Seller_Type', 'Transmission']
for col in cat_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

# %%
# Визуализируем распределение категориальных переменных
fig, axes = plt.subplots(2,2, figsize=(14,8))
sns.countplot(x='Fuel_Type', data=df, ax=axes[0,0])
sns.countplot(x='Seller_Type', data=df, ax=axes[0,1])
sns.countplot(x='Transmission', data=df, ax=axes[1,0])
axes[1,1].axis("off")
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 4. Анализ числовых переменных и аномальных значений

# %%
num_cols = ['Year','Selling_Price','Present_Price','Kms_Driven','Owner']
df[num_cols].describe()

# %%
# Гистограммы распределений
plt.figure(figsize=(12,8))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2,3,i)
    sns.histplot(df[col], kde=True, bins=20)
    plt.title(f"Распределение {col}")
plt.tight_layout()
plt.show()

# %% [markdown]
# ### Проверка выбросов через «ящики с усами»

# %%
plt.figure(figsize=(14,6))
sns.boxplot(data=df[num_cols])
plt.title("Boxplot всех числовых признаков")
plt.show()

# %% [markdown]
# **Наблюдение:**  
# - `Kms_Driven` и `Present_Price` содержат выбросы.  
# - `Year` можно трансформировать в возраст автомобиля.  

# %% [markdown]
# ## 5. Обработка аномальных значений

# %%
# Преобразуем Year в возраст автомобиля
current_year = 2024
df['Car_Age'] = current_year - df['Year']
df.drop('Year', axis=1, inplace=True)

# %%
# Удалим экстремальные выбросы по Kms_Driven с помощью межквартильного размаха (IQR)
Q1 = df['Kms_Driven'].quantile(0.25)
Q3 = df['Kms_Driven'].quantile(0.75)
IQR = Q3 - Q1
low, high = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

df = df[(df['Kms_Driven'] >= low) & (df['Kms_Driven'] <= high)]
print("Размер после удаления выбросов:", df.shape)

# %%
# Проверим на диаграмме после очистки
plt.figure(figsize=(8,5))
sns.boxplot(x=df['Kms_Driven'])
plt.title("Kms_Driven после очистки выбросов")
plt.show()

# %% [markdown]
# ## 6. Проверка корреляции числовых признаков

# %%
corr = df.corr(numeric_only=True)
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корреляция между числовыми признаками")
plt.show()

# %% [markdown]
# ## 7. Кодирование категориальных переменных

# %%
df_encoded = pd.get_dummies(df, columns=['Fuel_Type','Seller_Type','Transmission'], drop_first=True)
df_encoded.head()

# %% [markdown]
# ## 8. Нормализация числовых признаков

# %%
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_cols = ['Selling_Price','Present_Price','Kms_Driven','Car_Age']
df_encoded[scaled_cols] = scaler.fit_transform(df_encoded[scaled_cols])
df_encoded.head()

# %% [markdown]
# ## 9. Финальный анализ и подготовка к моделированию

# %%
print("Финальные столбцы:", df_encoded.columns.tolist())
df_encoded.info()

# %%
# Проверим итоговые распределения
sns.pairplot(df_encoded[['Selling_Price','Present_Price','Kms_Driven','Car_Age']])
plt.show()

# %% [markdown]
# ## Вывод
# 
# В результате проделанной предобработки:
# - Проведена проверка типов данных и дубликатов  
# - Удалены выбр


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB
